In [1]:
from lxml import etree
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import numpy as np
import requests


options = Options()
options.add_argument("--headless")
#options.add_argument("--start-maximized")
#options.add_argument("--disable-notifications")
options.add_argument("--incognito")


# Functions

In [21]:
def page_source_from_selenium(url):
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)


    total_height = int(driver.execute_script("return document.body.scrollHeight"))

    for i in range(1, total_height, 5):
        driver.execute_script("window.scrollTo(0, {});".format(i))

    page_source=driver.page_source


    print("Done Exporting Data")
    driver.close()
    driver.quit()
    print("Closed Driver")
    return page_source

def scrape_features_from_page(url):
    page_source=page_source_from_selenium(url)
    soup=bs4.BeautifulSoup(page_source,'lxml')
    dom = etree.HTML(str(soup))
    
    
    property_type=[i.text for i in dom.xpath("//span[@class='e2f34d59b1']") ][0]
    if property_type!="Hotel":
        print ("Non-Hotel- Skipping Link")
        return pd.DataFrame()
    
    else:
        hotel_name=[i.text for i in dom.xpath("//h2[@class='d2fee87262 pp-header__title']") ][0]
        _stars=[i for i in dom.xpath("//span[@data-testid='rating-circles']") ]
        stars=len(_stars[0].findall("span[@aria-hidden]"))
        location=[i.text.replace("\n","") for i in dom.xpath("//span[contains(@class, 'address')]")][0]
        review_rating=[i.text.replace("\n","") for i in dom.xpath("//div[contains(@aria-label, 'Scored')]")][0]
        location_score=[i.text.replace("\n","") for i in dom.xpath("//span[@class='review-score-badge']")][0]
        descriptions=[i.findall('p') for i in dom.xpath("//div[@id='property_description_content']")]
        description=" ".join([i.text for i in descriptions[0]])

        main_facilities=[]
        facilities_=[i for i in dom.xpath("//div[@class='hp_desc_important_facilities clearfix hp_desc_important_facilities--bui']")]

        for i in facilities_:
            #print(i.findall("div"))
            for ii in i.findall("div"):
                main_facilities.append(ii.get("data-name-en"))

        total_reviews=[i.text for i in dom.xpath("//div[@class='d8eab2cf7f c90c0a70d3 db63693c62']")][0].split(" ")[0]
        sub_ratings=[i.text for i in dom.xpath("//div[@class='ee746850b6 b8eef6afe1']")][:7]
        sub_ratings_categories=[i.text for i in dom.xpath("//span[@class='d6d4671780']")][:7]
        print(len(sub_ratings),len(sub_ratings_categories))

        sub_ratings_dict={}
        for i in range(len(sub_ratings)):
            #print(sub_ratings[i])
            #print(sub_ratings_categories[i])
            sub_ratings_dict[sub_ratings_categories[i]] = sub_ratings[i]

        #-----------------------------------------------    
        hotel_surroundings=[i.text for i in dom.xpath("//div[@class='b1e6dd8416 aacd9d0b0a']|//span[@class='b6f930dcc9']") if i.text is not None]
        hotel_surroundings_distance=[i.text for i in dom.xpath("//div[@class='db29ecfbe2 c90c0a70d3']") if i.text is not None]

        print("surroundings qc")
        print(len(hotel_surroundings), len(hotel_surroundings_distance))
        surroundings_dict = {hotel_surroundings[i]: hotel_surroundings_distance[i] for i in range(len(hotel_surroundings))}

        room_type  = [i.text.replace("\n","") for i in dom.xpath("//span[@class='hprt-roomtype-icon-link']")]


        price_list=[float(i.text.replace("\n","").strip().replace(u'₱\xa0', '').replace(",",""))
         for i in dom.xpath("//span[@class='prco-valign-middle-helper']")]

        cheapest_price=min(price_list)
        room_type_dict = {room_type[i]: price_list[i] for i in range(len(room_type))}

        facilities_groups=[i.replace("\n","") for i in dom.xpath("//div[@class='bui-title__text hotel-facilities-group__title-text']//text()") if (i!='\n') & (i.replace("\n","")!="Internet")]
        hfg=dom.xpath("//div[@class='bui-spacer--large']")
        all_facilities=[]
        for i in hfg:
            data=i.findall("div//div//div")
            facilities_per_group=[i.text.replace("\n","") for i in data if (i.text!="\n") & (i.text!=None)]
            if facilities_per_group !=[]:
                all_facilities.append(facilities_per_group)
            #print("")

        print("facilities qc")
        print(len(facilities_groups), len(all_facilities))
        facilities_dict={facilities_groups[i]: all_facilities[i] for i in range(len(facilities_groups))}
        internet_desc=[i.text.replace("\n","") for i in dom.xpath("//div[@class='bui-spacer--medium hotel-facilities-group__policy']") if ("wifi" in (i.text.lower()))]
        facilities_dict["Internet"]=internet_desc



        #-----------------------------------------------   
        all_features={"hotel_name_":hotel_name,"stars":stars,
                  "location":location , "location_score":location_score, 
                  "review_rating":review_rating, "description":description,
                  "main_facilities":main_facilities, "total_reviews":total_reviews,
                  "sub_ratings":sub_ratings, "sub_ratings_categories":sub_ratings_categories,"sub_ratings_dict":sub_ratings_dict,
                  "hotel_surroundings":hotel_surroundings, "hotel_surroundings_distance":hotel_surroundings_distance,"surroundings_dict":surroundings_dict,
                  "room_type":room_type, "price_list":price_list,"cheapeast_price":cheapest_price, "room_type_dict":room_type_dict,
                  "facilities_groups":facilities_groups,"all_facilities":all_facilities,"facilities_dict":facilities_dict         
    }


        df_all_features=pd.DataFrame({k: pd.Series([v]) for k,v in all_features.items()})
        return df_all_features


# Webscrape per page

In [4]:
df=pd.read_csv("../manila_url2.csv", nrows=2)
# links=df.links
df

,url
0,https://www.booking.com/hotel/ph/midas-hotel-a...
1,https://www.booking.com/hotel/ph/sofitel-phili...


In [22]:
from tqdm import tqdm
df=pd.read_csv("data/all_manila_hotels.csv",nrows = 2)
links=df.links

df_hotels_consolidated=pd.DataFrame()

start_index=0
end_index=10

for count,i in tqdm(enumerate(links[start_index:end_index])):
    print("Opening link- "+ str(count) + " " + str(i))
    print(df.hotel_name[count+start_index])
    try:
        df_to_append=scrape_features_from_page(i)
        df_to_append["link"]=links[count+start_index]
        df_to_append["hotel_name_from_all_urls"]=df.hotel_name[count+start_index]
        df_to_append["location_from_all_urls"]=df.location[count+start_index]
        df_to_append["distance_from_centre"]=df.distance[count+start_index]
        df_hotels_consolidated=pd.concat([df_hotels_consolidated,df_to_append])
    except Exception as e: 
        print("")
        print("Error:")
        print(str(e))
        print("Skipping due to error")
        continue
        
    print(" ")

0it [00:00, ?it/s]

Opening link- 0 https://www.booking.com/hotel/ph/diamond-philippines.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALP2NGbBsACAdICJGRiYTg5ODViLTY0OWYtNDJmMC05MmQ2LTQzYTNhOTM0OGMzNtgCBeACAQ&sid=62995f74fed87079ecbbad21978f8cb9&aid=304142&ucfs=1&arphpl=1&checkin=2022-11-25&checkout=2022-11-26&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=e9da55d35eaa04a2&srepoch=1668773545&all_sr_blocks=29318601_0_2_1_0&highlighted_blocks=29318601_0_2_1_0&matching_block_id=29318601_0_2_1_0&sr_pri_blocks=29318601_0_2_1_0__15305&tpi_r=1&from=searchresults#hotelTmpl
Diamond Hotel Philippines - Multiple Use Hotel
Done Exporting Data
Closed Driver


1it [00:49, 49.42s/it]

7 7
surroundings qc
33 33
facilities qc
19 19
 
Opening link- 1 https://www.booking.com/hotel/ph/peaceful-and-beachy-staycation-azure.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALP2NGbBsACAdICJGRiYTg5ODViLTY0OWYtNDJmMC05MmQ2LTQzYTNhOTM0OGMzNtgCBeACAQ&sid=62995f74fed87079ecbbad21978f8cb9&aid=304142&ucfs=1&arphpl=1&checkin=2022-11-25&checkout=2022-11-26&dest_id=-2437894&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr_order=popularity&srpvid=e9da55d35eaa04a2&srepoch=1668773545&all_sr_blocks=399263001_202857528_4_0_0&highlighted_blocks=399263001_202857528_4_0_0&matching_block_id=399263001_202857528_4_0_0&sr_pri_blocks=399263001_202857528_4_0_0__185980&tpi_r=2&from=searchresults#hotelTmpl
Azure Peaceful and Beachy Staycation
Done Exporting Data
Closed Driver


2it [01:30, 45.44s/it]

Non-Hotel- Skipping Link
 


In [24]:
df_hotels_consolidated#['room_type'][0]

,hotel_name_,stars,location,location_score,review_rating,description,main_facilities,total_reviews,sub_ratings,sub_ratings_categories,...,price_list,cheapeast_price,room_type_dict,facilities_groups,all_facilities,facilities_dict,link,hotel_name_from_all_urls,location_from_all_urls,distance_from_centre
0,Diamond Hotel Philippines - Multiple Use Hotel,5.0,"Dr. J. Quintos cor. Roxas Blvd Malate, 1000 Ma...",8.5,7.9,"Located along Manila Bay, a 30-minute drive fr...","[Swimming pool, Non Smoking Rooms, Fitness Roo...",209,"[8.3, 8.0, 8.2, 8.3, 7.4, 8.5, 7.3]","[Staff, Facilities, Cleanliness, Comfort, Valu...",...,"[6300.0, 12274.0, 11733.0, 12274.0, 11733.0]",6300.0,"{'Deluxe Regency Non-Smoking': 6300.0, 'Diamon...","[Bathroom, Bedroom, View, Outdoors, Kitchen, R...","[[Toilet paper, Towels, Bidet, Additional toil...","{'Bathroom': ['Toilet paper', 'Towels', 'Bidet...",https://www.booking.com/hotel/ph/diamond-phili...,Diamond Hotel Philippines - Multiple Use Hotel,"Manila Bay, Manila",2.7 km from centre


In [30]:
import datetime

In [60]:
today =  datetime.datetime.strftime(datetime.date.today(), "%Y-%m-%d")
today

'2022-11-23'

In [64]:
days_from_today = 3
length_of_stay = 1

In [65]:
start_date = datetime.datetime.strptime(today, "%Y-%m-%d") + datetime.timedelta(days=days_from_today)
start_date = start_date.strftime("%Y-%m-%d")
start_date

'2022-11-26'

In [69]:
end_date = datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days=length_of_stay)
end_date = end_date.strftime("%Y-%m-%d")
end_date

'2022-11-27'

In [70]:
"https://www.booking.com/searchresults.en-gb.html?ss=Cebu&ssne=Baguio&ssne_untouched=Baguio&label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&checkin=" + start_date + "&checkout=" + end_date + "&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"


'https://www.booking.com/searchresults.en-gb.html?ss=Cebu&ssne=Baguio&ssne_untouched=Baguio&label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&checkin=2022-11-26&checkout=2022-11-27&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

In [71]:
"https://www.booking.com/searchresults.en-gb.html?ss=Cebu&ssne=Baguio&ssne_untouched=Baguio&label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&checkin=2022-11-25&checkout=2022-11-26&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"


'https://www.booking.com/searchresults.en-gb.html?ss=Cebu&ssne=Baguio&ssne_untouched=Baguio&label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&checkin=2022-11-25&checkout=2022-11-26&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

In [ ]:
target_page = "https://www.booking.com/searchresults.en-gb.html?ss=Cebu&ssne=Baguio&ssne_untouched=Baguio&label=gen173nr-1FCAEoggI46AdIM1gEaLQBiAEBmAEJuAEXyAEM2AEB6AEB-AELiAIBqAIDuALa9-ebBsACAdICJDNiMWUxMTlhLTc3ODEtNGI3NC05MWVlLTQ5MDA0MDQzYzY5NNgCBuACAQ&sid=b3a20c12be6b3d4d9b1b0a71b11d8898&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&checkin=2022-11-25&checkout=2022-11-26&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"
